# Configuration Patterns

This notebook explores configuration options: Python objects, `from_dict()`, multiple stores sharing a backend, and config validation.

In [ ]:
import tempfile

from remote_store import BackendConfig, Registry, RegistryConfig, StoreProfile

## Config-as-Code

Build configuration directly with Python objects. This is the most type-safe approach.

In [ ]:
tmpdir = tempfile.mkdtemp()

config = RegistryConfig(
    backends={
        "local": BackendConfig(type="local", options={"root": tmpdir}),
    },
    stores={
        "uploads": StoreProfile(backend="local", root_path="uploads"),
        "reports": StoreProfile(backend="local", root_path="reports"),
        "archive": StoreProfile(backend="local", root_path="archive"),
    },
)
print(f"Backends: {list(config.backends.keys())}")
print(f"Stores: {list(config.stores.keys())}")

## Multiple Stores, One Backend

All three stores share the same local backend but are scoped to different root paths. They operate independently.

In [ ]:
registry = Registry(config)

uploads = registry.get_store("uploads")
reports = registry.get_store("reports")

uploads.write("images/photo.jpg", b"fake-image-data")
reports.write("quarterly/q4.csv", b"revenue,cost\n100,80")

print("Uploads:", [f.name for f in uploads.list_files("images")])
print("Reports:", [f.name for f in reports.list_files("quarterly")])

# Stores are isolated — uploads can't see reports' files
print(f"\nuploads sees quarterly/q4.csv? {uploads.exists('quarterly/q4.csv')}")
print(f"reports sees images/photo.jpg? {reports.exists('images/photo.jpg')}")

registry.close()

## from_dict()

Parse configuration from a plain dict — useful when loading from TOML, JSON, or YAML files.

In [ ]:
tmpdir2 = tempfile.mkdtemp()

raw_config = {
    "backends": {
        "local": {"type": "local", "options": {"root": tmpdir2}},
    },
    "stores": {
        "data": {"backend": "local", "root_path": "data"},
        "logs": {"backend": "local", "root_path": "logs"},
    },
}

config2 = RegistryConfig.from_dict(raw_config)
print(f"Parsed config: {config2}")

with Registry(config2) as reg:
    data = reg.get_store("data")
    data.write("test.txt", b"from_dict works!")
    print(f"Content: {data.read_bytes('test.txt').decode()}")

## Config Validation

The config validates that all store profiles reference existing backends.

In [ ]:
try:
    bad_config = RegistryConfig(
        backends={},
        stores={"orphan": StoreProfile(backend="nonexistent")},
    )
    bad_config.validate()
except ValueError as exc:
    print(f"Validation error: {exc}")

## Cleanup

In [ ]:
import shutil

shutil.rmtree(tmpdir)
shutil.rmtree(tmpdir2)
print("Cleaned up.")